In [1]:
#Importing all required Modules

from sqlalchemy.engine import create_engine
import pandas as pd
import datetime
from sqlalchemy.dialects.oracle import DATE, VARCHAR2, FLOAT, NUMBER,TIMESTAMP
from sqlalchemy import text,String,types
from datetime import datetime

In [4]:
# Extracting data from Source

with source_engine.begin():
    try:
        query = ('SELECT distinct predictedcategory FROM fastpace.predictionresults')
        predicted_category = pd.read_sql(query, source_engine)
        input_predicted_category = list(predicted_category['predictedcategory'])
    except Exception as e:
        print('Error: ',e)

#predicted_category
#input_predicted_category

In [5]:
#creating empty dataframe

visit_category_dim_df = pd.DataFrame(columns=['visit_category_dim_id', 'visit_category_code', 'visit_category_desc', 'created_date'])
#visit_category_dim_df

In [6]:
#Loading the data into dataframe

visit_category_dim_df['visit_category_desc'] = input_predicted_category

In [7]:
#Defining the data types

data_types = {'visit_category_dim_id': types.Integer(),
              'visit_category_code':  types.String(255),
              'visit_category_desc':  types.String(255),
              'created_date': types.Date()}

In [8]:
try:
 visit_category_dim_df.to_sql('visit_category_dim',staging_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into Staging_DB')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into Staging_DB


##ETL from Staging to Datawarehouse

In [17]:
#Extracting data from staging DB

with staging_engine.begin():
 try:
     query = ("select distinct visit_category_desc from staging_source.visit_category_dim;")
     prediction_results_df = pd.read_sql_query(query, staging_engine)
     input_categories = list(prediction_results_df['visit_category_desc'])
 except Exception as e:
     print("An error occurred:", e)

In [18]:
#Creating empty dataframe

visit_category_dim_df = pd.DataFrame(columns=['visit_category_dim_id', 'visit_category_code', 'visit_category_desc', 'created_date'])

In [19]:
#Determining the current maximum ID in the DataFrame, 
current_max_id = visit_category_dim_df['visit_category_dim_id'].max() if not visit_category_dim_df.empty else 0
# Using pandas Series to generate IDs
new_ids = pd.Series(range(current_max_id + 1, current_max_id + 1 + len(input_categories)))


In [20]:
#TRANSFORMATION 

new_records = pd.DataFrame({
 'visit_category_dim_id': new_ids,
 'visit_category_code': [category[:3].upper() for category in input_categories],
 'visit_category_desc': input_categories,
 'created_date': [datetime.now().strftime('%Y-%m-%d')] * len(input_categories)
})

In [21]:
# Appending the new records to the original DataFrame

visit_category_dim_df = pd.concat([visit_category_dim_df, new_records], ignore_index=True)

In [22]:
#Defining the data types 

data_types = {
 'visit_category_dim_id': types.Integer(),
 'visit_category_code': types.String(length=3),
 'visit_category_desc': types.String(),
 'created_date': types.String(length=10) # storing dates as strings in YYYY-MM-DD format
}

In [23]:
#Loading the data into Datawarehouse

try:
 visit_category_dim_df.to_sql('visit_category_dim',datawarehouse_engine, if_exists='append', index=False, dtype= data_types)
 print('successfully loaded into DataWarehouse')
except Exception as e:
 print("An error occurred:", e)

successfully loaded into DataWarehouse
